In [38]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round
from decimal import Decimal, ROUND_HALF_UP

spark = SparkSession.builder.appName("AcquirerLTDA").getOrCreate()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 2s (138 kB/s)
Reading package lists... Done


In [39]:
# Transactions and Contracts data
transactions = [
      {"transaction_id": 1, "client_id": 3545, "total_amount": 3000, "discount_percentage": 6.99},
      {"transaction_id": 2, "client_id": 3545, "total_amount": 4500, "discount_percentage": 0.45},
      {"transaction_id": 3, "client_id": 3509, "total_amount": 69998, "discount_percentage": 0},
      {"transaction_id": 4, "client_id": 3510, "total_amount": 1, "discount_percentage": None},
      {"transaction_id": 5, "client_id": 4510, "total_amount": 34, "discount_percentage": 40},
  ]

contracts = [
      {"contract_id": 3, "client_id": 3545, "client_name": "Magazine Luana", "percentage": 2.00, "is_active": True},
      {"contract_id": 4, "client_id": 3545, "client_name": "Magazine Luana", "percentage": 1.95, "is_active": False},
      {"contract_id": 5, "client_id": 3509, "client_name": "Lojas Italianas", "percentage": 1, "is_active": True},
      {"contract_id": 6, "client_id": 3510, "client_name": "Carrerfive", "percentage": 3.00, "is_active": True},
  ]

# Ajustar None dos campos discount_percentage e percentage
for transaction in transactions:
    transaction["discount_percentage"] = Decimal(transaction["discount_percentage"]) if transaction["discount_percentage"] is not None else Decimal(0)

for contract in contracts:
    contract["percentage"] = Decimal(contract["percentage"]) if contract["percentage"] is not None else Decimal(0)

# Converter os dados em DataFrames do Spark
transactions_df = spark.createDataFrame(transactions)
contracts_df = spark.createDataFrame(contracts)


In [40]:
# Calculo do ganho total
profit = (col("total_amount") - (col("total_amount") * (col("discount_percentage") / 100))) * (col("percentage") / 100)
profit_sum = transactions_df.join(contracts_df, transactions_df.client_id == contracts_df.client_id, "inner") \
    .filter(col("is_active") == True) \
    .select(profit.alias("profit")) \
    .groupBy().sum() \
    .collect()[0][0]

# Arredondar o resultado com precisão decimal após coletar o resultado
rounded_profit = Decimal(str(profit_sum)).quantize(Decimal("0.000"), rounding=ROUND_HALF_UP)

In [42]:
print(f"O ganho total da empresa Acquirer LTDA é: {rounded_profit}")

# Encerrar a sessão do Spark
spark.stop()

O ganho total da empresa Acquirer LTDA é: 845.411
